<a href="https://colab.research.google.com/github/hood-boi/world-news-chatbot/blob/master/World_News_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Extraction

In [0]:
import requests
import json
import time

In [0]:
!pip install langdetect

     |████████████████████████████████| 1.0MB 6.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/ec/0c/a9/1647275e7ef5014e7b83ff30105180e332867d65e7617ddafe
Successfully built langdetect


In [0]:
from langdetect import detect

In [0]:
# -*- coding: utf-8 -*-
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [0]:
def updateListingDict(postsJSON, postsDICT):
    chunk_size = len(postsJSON["data"]["children"])
    posts = postsJSON["data"]["children"]
    
    for i in range(chunk_size): #Loop through each reddit post
        r_post_title = posts[i]["data"]["title"];
        r_post_title = " ".join(r_post_title.split());
        r_post_title = r_post_title.replace('\n', ' ');
        r_post_title = r_post_title.replace('\t', ' ');
        r_post_title = r_post_title.replace('\'', '');
        r_post_title = r_post_title.replace('\"', '');
        
        postsDICT[posts[i]["data"]["id"]] = r_post_title;

        
    

after = '';
listing_dict = {};

for i in range(0, 10):
    if(after):
        print("[AFTER] = ", after)
        r_worldnews = requests.get('https://www.reddit.com/r/worldnews/top.json?t=all&limit=100&after='+after, 
                                headers = { 'User-agent' : 'ChatBot', 
                                            'Accept' : '*/*',
                                            'Cache-Control' : 'no-cache'
                                          });
        if(r_worldnews.status_code == 200):
            postsJSON = r_worldnews.json()
            updateListingDict(postsJSON, listing_dict);
            after = postsJSON["data"]["after"];
    else:
        print("[AFTER] = NONE")
        r_worldnews = requests.get('https://www.reddit.com/r/worldnews/top.json?t=all&limit=100', 
                                headers = { 'User-agent' : 'ChatBot', 
                                            'Accept' : '*/*',
                                            'Cache-Control' : 'no-cache' 
                                          });
        if(r_worldnews.status_code == 200):
            postsJSON = r_worldnews.json()
            updateListingDict(postsJSON, listing_dict);
            #print("[GET] request successful")
            after = postsJSON["data"]["after"];
            
            


[AFTER] = NONE
[AFTER] =  t3_8cmonj
[AFTER] =  t3_ajo3wy
[AFTER] =  t3_5d2cwf
[AFTER] =  t3_51l6zy
[AFTER] =  t3_aek7g4
[AFTER] =  t3_7cirxj
[AFTER] =  t3_6ho8wq
[AFTER] =  t3_ajg0p5
[AFTER] =  t3_6y0tpf


In [0]:
print(len(listing_dict))

for key, value in listing_dict.items():
    print(key, value)
    break;

82
901p5f Two weeks before his inauguration, Donald J. Trump was shown highly classified intelligence indicating that President Vladimir V. Putin of Russia had personally ordered complex cyberattacks to sway the 2016 American election


In [0]:
print(key,value)   

901p5f Two weeks before his inauguration, Donald J. Trump was shown highly classified intelligence indicating that President Vladimir V. Putin of Russia had personally ordered complex cyberattacks to sway the 2016 American election


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
#4/igGrpBUzgmmOTdhG77m8_Ej4eL8V8wcgV1kMDSd2zZel0dhlp_w-eek

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
! mkdir /content/gdrive/My\ Drive/Chatbot

mkdir: cannot create directory ‘/content/gdrive/My Drive/Chatbot’: File exists


In [0]:
! rm /content/gdrive/My\ Drive/Chatbot/rWorldNews.txt

rm: cannot remove '/content/gdrive/My Drive/Chatbot/rWorldNews.txt': No such file or directory


In [0]:
file_handle = open('/content/gdrive/My Drive/Chatbot/rWorldNews.txt', 'a+')

#Expecting array of JSON children
def recursePostComments(context, commentsJSON):
   
    size = len(commentsJSON)
    for i in range(0, size):
        #print(commentsJSON[i])
        if('data' in commentsJSON[i].keys()):
            if('body' in commentsJSON[i]['data'].keys()):
                reply = commentsJSON[i]['data']['body'];
                if("[deleted]" in reply or "[removed]" in reply):
                    continue;
                else:
                    reply = reply.replace('\n', ' ');
                    reply = reply.replace('\t', ' ');
                    reply = reply.replace('\'', '');
                    reply = reply.replace('\"', '');
                    reply = reply.replace('&gt', '');
                    reply = reply.replace('&lt', '');
                    #print("Context : ", context);
                    
                    #if (len(reply) > 500):
                        #print("\tLen :", len(reply));
                    #    t_reply = summarizeText(reply);
                    #    if (len(t_reply) > 0):
                    #        reply = t_reply;
                    #print("\tReply :", reply);
                    
                    context.replace('\t', ' ');
                    
                    reply.replace('\t', ' ');
                    reply = " ".join(reply.split());
                    
                    if("Article has nothing" in context):
                        print("NIqqa")
                    
                    if(reply.isspace() or context.isspace()):
                        print("Error : ", reply, ", ", context)
                        continue;                    
                    else:
                        line_to_write = "{}\t{}\n".format(reply.strip(), context.strip());
                        file_handle.write(line_to_write)
                    
                    if('replies' in commentsJSON[i]['data'].keys()):
                        if(commentsJSON[i]['data']['replies']):
                            if('data' in commentsJSON[i]['data']['replies'].keys()):
                                if(commentsJSON[i]['data']['replies']['data']):
                                    if('children' in commentsJSON[i]['data']['replies']['data'].keys()):
                                        recursePostComments(reply, commentsJSON[i]['data']['replies']['data']['children']);
            else:
                continue;
        else:
            continue;
    return;



In [0]:
for key, value in listing_dict.items():
    print(key, value)
    r_comments = requests.get('https://www.reddit.com/r/worldnews/comments/' + key +'.json?limit=10000&sort=confidence&depth=4', 
                                headers = { 'User-agent' : 'ChatBot', 
                                            'Accept' : '*/*',
                                            'Cache-Control' : 'no-cache' 
                                          });
    r_comments_json = r_comments.json();
    context = value;
    commentsJSON = r_comments_json[1]["data"]["children"]
    recursePostComments(context, commentsJSON)
    
file_handle.close()

#Overfitting and Testing Generative model

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

import torchtext
import random


In [0]:
text_field = torchtext.data.Field(sequential=True,      # text sequence
                                  tokenize=lambda x: x, # because are building a character-RNN
                                  include_lengths=True, # to track the length of sequences, for batching
                                  batch_first=True,
                                  use_vocab=True
                                 )       # to turn each character into an integer index
label_field = torchtext.data.Field(sequential=True,    # text sequence
                                   use_vocab=True,     # don't need to track vocabulary
                                   is_target=True,      
                                   batch_first=True,
                                   tokenize=lambda x: x,
                                   preprocessing=lambda x: x,
                                   init_token="<BOS>",
                                   eos_token="<EOS>"
                                  ) 

fields = [('reply', label_field), ('context', text_field)]
dataset = torchtext.data.TabularDataset("/content/gdrive/My Drive/Chatbot/rWorldNews.txt", # name of the file
                                        "tsv",               # fields are separated by a tab
                                        fields)

In [4]:
len(dataset)

424044

In [0]:
train = torchtext.data.Dataset(dataset, fields)

In [6]:
len(train)

424044

In [0]:
text_field.build_vocab(train)
label_field.build_vocab(train)

In [8]:
print(text_field.vocab.itos)
print(label_field.vocab.itos)

['<unk>', '<pad>', ' ', 'e', 't', 'a', 'o', 'i', 'n', 's', 'r', 'h', 'l', 'd', 'u', 'c', 'm', 'p', 'g', 'y', 'w', 'f', 'b', '.', 'v', 'k', ',', 'I', '-', '/', 'T', 'S', 'A', '0', '1', 'C', 'x', '*', ':', '2', 'j', 'P', 'M', 'W', 'R', 'E', 'N', '?', 'B', 'H', 'U', ')', 'D', '(', 'O', 'F', '’', 'z', ';', '5', '3', 'G', '_', '8', '9', '6', 'L', '4', '7', 'K', 'q', ']', '[', 'Y', 'J', '!', 'V', '%', '“', '”', '=', '$', '&', '#', '~', '^', '|', 'Q', 'Z', '‘', '—', 'X', '+', '\\', '–', '@', 'é', '£', '…', 'о', 'ü', 'е', '€', '•', '⠀', 'ó', 'и', 'н', '️', 'а', 'т', 'á', 'ñ', 'с', '°', '・', 'ä', 'л', 'р', 'ا', '§', '\u200b', 'м', 'ы', 'в', '`', 'к', 'ن', 'д', '´', '✔', '⣿', 'п', 'í', 'ی', 'ö', '，', 'я', '😂', 'б', 'у', '。', '🤔', '\u200d', '💦', 'ر', '¥', 'م', 'ه', 'の', 'が', 'ч', 'º', '🇧', 'و', '¯', '·', '■', 'Ä', '‚', 'ل', '¶', 'た', 'な', '🇬', 'د', 'ت', '👍', 'г', '不', 'か', 'て', 'ب', 'っ', '人', '★', '👏', '、', 'い', '☆', 'ğ', 'Ü', 'ь', '™', '一', 'з', '💘', '₂', '交', '全', '！', '🇺', 'й', '⠛', '⌐', 'ツ', 

In [0]:
train_iter = torchtext.data.BucketIterator(train,
                                           batch_size=1,
                                           sort_key=lambda x: len(x.context), # to minimize padding
                                           sort_within_batch=True,        # sort within each batch
                                           repeat=False,
                                          )                  # repeat the iterator for many epochs

In [10]:
k = 0;
for batch in train_iter:
    
    #max_length_seq = batch.context[0].shape[1];
    #print("-----Batch #", k+1, "-----");
    #print("Maximum length of the input sequence : ", max_length_seq);
    #padding_per_batch = int(torch.sum(max_length_seq - batch.sms[1]))
    #print("Number of padding for batch : ", padding_per_batch)
    
    print("Batch Size : ", len(batch))
    #print("Batch Context : ", batch.context)
    print("Batch Context Shape: ", batch.context[0].shape)
    #print(batch.reply)
    print("Batch Context Shape: ", batch.reply.shape)
    k += 1;
    
    if(k >= 1):
        break;

Batch Size :  1
Batch Context Shape:  torch.Size([1, 113])
Batch Context Shape:  torch.Size([1, 118])


In [0]:

string_array =[];
for i in range(0, batch.reply.shape[1]):
    string_array.append(label_field.vocab.itos[(batch.reply[0][i])])
''.join(string_array)

'<BOS>I just don\'t get it. How do people think Trump is looking to help the little guy, aka us citizens? He wants to run the country like a business. He is basically only looking out for his wallet and his own ass.  He has praised dictators and put Putin on a pedestal above all else.  He has unethically benefited from a business standpoint by having the government spend millions at his hotels.   He has refused to demonstrate sympathy when a tragic shooting occurs.  He has mocked gold star families.  He goes on tirades on Twitter.  He fabricates "facts" (like the ones related to the Queen) to make himself seem better.  He had a scandal with a porn star.  He has literally been caught saying grab em by the pussy (and said some incestuous stuff about his daughter).  He constantly attacks the Constitution of Free Speech by calling various journalism "Fake News".   He has ruined our relationships with previous allies.  He tried to start a trade war.  His tax cuts only helped large corporati

In [11]:
input_vocab_size = len(text_field.vocab.itos)
reply_vocab_size = len(label_field.vocab.itos)
print("Input Vocab Size: ", input_vocab_size)
print("Reply Vocab size: ", reply_vocab_size)

Input Vocab Size:  1275
Reply Vocab size:  2232


In [0]:
class ChatBot(nn.Module):
    def __init__(self, vocab_size, hidden_size, n_layers=1):
        super(ChatBot, self).__init__()

        # identiy matrix for generating one-hot vectors
        self.ident = torch.eye(vocab_size)

        # recurrent neural network
        self.rnn = nn.GRU(vocab_size, hidden_size, n_layers, batch_first=True)

        # a fully-connect layer that outputs a distribution over
        # the next token, given the RNN output
        self.decoder = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, inp, hidden=None):
        inp = self.ident[inp]                  # generate one-hot vectors of input
        output, hidden = self.rnn(inp, hidden) # get the next output and hidden state
        output = self.decoder(output)          # predict distribution over next tokens
        return output, hidden

In [0]:
model = ChatBot(vocab_size, 1024)


In [0]:
criterion = nn.CrossEntropyLoss()

In [0]:
bos_input = torch.Tensor([label_field.vocab.stoi["<BOS>"]]).long().unsqueeze(0)
output, hidden = model(bos_input, hidden=None)

In [0]:
batch.reply[0][1]

tensor(29)

In [0]:
target = torch.Tensor([batch.reply[0][1]]).long().unsqueeze(0);

tensor([[29]])

In [0]:
print(output.shape)
print(target.shape)

torch.Size([1, 1, 95])
torch.Size([1, 1])


In [0]:
criterion(output.reshape(-1, vocab_size), target.reshape(-1))

tensor(4.5767, grad_fn=<NllLossBackward>)

In [0]:
class ChatBot(nn.Module):
    def __init__(self, context_vocab_size, encoder_hidden_size, reply_vocab_size, generator_hidden_size, encoder_layers = 1, generator_layers = 1):
        super(ChatBot, self).__init__()

        # identiy matrix for generating one-hot vectors
        self.ident_context = torch.eye(context_vocab_size)
        self.ident_reply = torch.eye(reply_vocab_size)

        # Left side of the model which encodes the context. We will use the final hidden layer input to generator RNN
        self.encoderRnn = nn.GRU(context_vocab_size, encoder_hidden_size, encoder_layers, batch_first=True)
        
        # Generator RNN
        self.generatorRNN = nn.GRU(reply_vocab_size, generator_hidden_size, generator_layers, batch_first=True)
        

        # a fully-connect layer that outputs a distribution over
        # the next token, given the RNN output
        self.decoder = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, context, response, hidden=None):
        context = self.ident_context[context]                  # generate one-hot vectors of input
        response = self.ident_reply[response]
        output, hidden = self.rnn(inp, hidden) # get the next output and hidden state
        output = self.decoder(output)          # predict distribution over next tokens
        return output, hidden

In [0]:
ident_context = torch.eye(input_vocab_size)
ident_reply = torch.eye(reply_vocab_size)
encoderRnn = nn.GRU(input_vocab_size, 100, 1, batch_first=True)
generatorRNN = nn.GRU(reply_vocab_size, 100, 1, batch_first=True)

In [0]:
batch.context[0].shape

torch.Size([1, 226])

In [0]:
batch.reply.shape #[0] is batch size [1] is sequence length

torch.Size([1, 1506])

In [0]:
icontext = ident_context[batch.context[0]]
out, hidden_encoder = encoderRnn(icontext, None)
print(out.shape)
print(hidden_encoder.shape)
ireply = ident_reply[batch.reply]
out2, hidden_generator = generatorRNN(ireply, hidden_encoder)
print(out2.shape)
print(hidden_generator.shape)

torch.Size([1, 226, 100])
torch.Size([1, 1, 100])
torch.Size([1, 1506, 100])
torch.Size([1, 1, 100])


In [0]:
class ChatBot(nn.Module):
    def __init__(self, context_vocab_size, encoder_hidden_size, reply_vocab_size, generator_hidden_size, encoder_layers = 1, generator_layers = 1):
        super(ChatBot, self).__init__()

        # identiy matrix for generating one-hot vectors
        self.ident_context = torch.eye(context_vocab_size)
        self.ident_reply = torch.eye(reply_vocab_size)

        # Left side of the model which encodes the context. We will use the final hidden layer input to generator RNN
        self.encoderRnn = nn.GRU(context_vocab_size, encoder_hidden_size, encoder_layers, batch_first=True)
        
        # Generator RNN
        self.generatorRNN = nn.GRU(reply_vocab_size, generator_hidden_size, generator_layers, batch_first=True)
        

        # a fully-connect layer that outputs a distribution over
        # the next token, given the RNN output
        self.decoder = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, context, response, hidden=None):
        context = self.ident_context[context]                  # generate one-hot vectors of input
        response = self.ident_reply[response]
        output, hidden = self.rnn(inp, hidden) # get the next output and hidden state
        output = self.decoder(output)          # predict distribution over next tokens
        return output, hidden